
<h1 align="center">Teste 04: classificação binária aedes com augmentation</h1>
<h2 align="center">Descritor: 40 MFCCs</h2>
<h2 align="center">Audios: 1368 por classe</h2>

In [27]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

In [2]:
def read_pickle(name):
    with (open(name, 'rb')) as openfile:
        while True:
            try:
                one_instance = pickle.load(openfile)
            except EOFError:
                break
    one_instance = np.asanyarray(one_instance)
    return one_instance

In [116]:
path = '/content/drive/MyDrive/pesquisa_mosquito/pickles/2_5s/teste_04/'
save = '/content/drive/MyDrive/pesquisa_mosquito/resultados/'

In [14]:
pickles = [x for x in os.listdir(path) if 'mfccs' in x]
pickles

['handcrafted_features_mfccs_aedes_binary.pickle',
 'handcrafted_labels_mfccs_aedes_binary.pickle']

In [15]:
x = read_pickle(path+pickles[0])
y = read_pickle(path+pickles[1])

In [16]:
print(x.shape,'\n',y.shape)

(2736, 40) 
 (2736,)


In [22]:
dfFeats = pd.DataFrame(x)
dfFeats['y'] = y
dfFeats = shuffle(dfFeats)
dfFeats

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,y
737,-646.346436,81.595123,15.031679,28.317318,8.352692,25.507996,5.694583,20.503420,7.144488,15.080863,...,2.361719,-9.487845,-5.152635,-0.699546,4.718816,6.982303,0.510184,2.765040,6.861376,1
683,-638.422241,86.895706,-1.960706,32.834991,2.696577,21.283878,1.281118,14.889467,8.537895,14.392237,...,-0.307040,-2.471482,0.167614,-1.909724,-1.169143,2.257909,0.219335,-3.542279,-5.412747,1
2152,-567.923401,81.059341,-8.602930,35.909405,3.815357,22.541832,-13.182562,18.526320,11.892356,26.540754,...,1.746213,-1.052255,1.434508,1.992522,-3.846825,0.111342,0.531464,0.195328,0.645491,0
969,-688.702820,63.160572,8.598861,40.403400,10.077956,17.271574,14.939156,8.861146,5.110004,9.830078,...,3.763705,7.687103,6.202844,3.677347,4.197947,2.406895,-1.000141,-0.978386,-0.465325,1
326,-636.616516,56.000069,17.844727,26.811686,18.337004,24.435457,18.001833,9.492730,12.108727,13.850323,...,1.302549,-0.686567,-1.974565,-0.287751,1.210859,0.400225,1.708349,2.665054,1.073632,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,-628.732056,83.002449,10.423815,25.374533,0.705394,23.078785,5.801353,24.293564,14.729966,19.120163,...,3.118835,4.574366,-0.172395,-0.503634,-2.851947,0.111961,2.279088,7.048489,1.757725,0
2389,-638.354553,89.048126,16.964251,32.109287,9.243767,23.053688,-0.258633,13.167319,6.324668,10.778255,...,5.567459,2.686037,-0.123313,2.278721,-1.442936,-1.419255,-0.197279,0.442958,-2.790438,0
1861,-670.816528,85.039024,13.056669,34.382614,-0.758199,20.631338,-14.483373,9.033955,-14.127332,11.315678,...,-8.025064,-11.603100,-3.992353,-5.608555,-2.630988,-4.863835,-4.315022,-10.361263,-2.293501,0
428,-599.163940,99.615051,-29.104984,35.302204,-4.476748,22.878214,9.133309,6.016124,5.776228,5.047007,...,3.475670,0.956509,-2.308736,-1.931965,-0.076604,-2.016298,2.117249,0.718277,-1.883780,1


In [23]:
dfFeats['y'].value_counts()

1    1368
0    1368
Name: y, dtype: int64

In [25]:
X = dfFeats.iloc[:,:40]
Y = dfFeats['y']
print(X.shape,'\n',Y.shape)

(2736, 40) 
 (2736,)


## Grid Seach

In [28]:
#para treinar o classificador
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=42)

In [29]:
#random forest
classifier = RandomForestClassifier()
param_grid = {'n_estimators': [90,100,110,120,150]}
search = GridSearchCV(classifier,
                      param_grid,
                      scoring = 'accuracy',
                      cv = 10,
                      n_jobs = -1)
search.fit(x_train, y_train)
print(search.best_params_)

{'n_estimators': 110}


In [32]:
param_rf = search.best_params_
param_rf

{'n_estimators': 110}

In [33]:
#svm
classifier = SVC()
param_grid = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'),'C':[0.1,0.5,0.7,1.0,1.2]}
search = GridSearchCV(classifier,
                      param_grid,
                      scoring = 'accuracy',cv=5,
                      n_jobs = -1)
search.fit(x_train,y_train)
print(search.best_params_)

{'C': 0.1, 'kernel': 'linear'}


In [34]:
param_svm = search.best_params_
param_svm

{'C': 0.1, 'kernel': 'linear'}

## Cross validation SVM

In [36]:
clf = SVC(C=param_svm['C'],kernel=param_svm['kernel'])

In [37]:
clf.fit(x_train, y_train)

SVC(C=0.1, kernel='linear')

In [38]:
pred = clf.predict(x_test)
accuracy_score(y_test, pred)

0.948905109489051

In [51]:
#adicionar as metricas precision, recall, f1 
metrics = ['accuracy', 'precision','recall','f1']

In [52]:
scores_svm = cross_validate(clf, X,Y, cv=10, scoring=metrics)

In [53]:
scores_svm

{'fit_time': array([0.70842862, 0.75051904, 0.82824993, 0.70837235, 0.72752142,
        0.80757689, 0.65226197, 0.79568243, 0.64819646, 0.58953905]),
 'score_time': array([0.00818682, 0.00826883, 0.00830293, 0.0080626 , 0.0082283 ,
        0.00818396, 0.00838375, 0.00825953, 0.00804925, 0.0081892 ]),
 'test_accuracy': array([0.94160584, 0.94525547, 0.93430657, 0.91605839, 0.9379562 ,
        0.9379562 , 0.94139194, 0.95604396, 0.92307692, 0.94871795]),
 'test_f1': array([0.94244604, 0.94545455, 0.93233083, 0.91385768, 0.93862816,
        0.93818182, 0.93939394, 0.95588235, 0.92193309, 0.94927536]),
 'test_precision': array([0.92907801, 0.94202899, 0.96124031, 0.93846154, 0.92857143,
        0.93478261, 0.97637795, 0.96296296, 0.93233083, 0.93571429]),
 'test_recall': array([0.95620438, 0.94890511, 0.90510949, 0.89051095, 0.94890511,
        0.94160584, 0.90510949, 0.94890511, 0.91176471, 0.96323529])}

In [54]:
#para calcular o kappa
kappa = make_scorer(cohen_kappa_score)

In [55]:
kappa_svm = cross_validate(clf, X, Y, cv=10, scoring=kappa)

In [56]:
kappa_svm

{'fit_time': array([0.66555214, 0.74853206, 0.81955934, 0.70866394, 0.72738695,
        0.81132436, 0.65442824, 0.78290176, 0.64777517, 0.59439492]),
 'score_time': array([0.0063858 , 0.00592518, 0.00586009, 0.00573206, 0.00590277,
        0.00588655, 0.00595927, 0.00607729, 0.00587344, 0.00610042]),
 'test_score': array([0.88321168, 0.89051095, 0.86861314, 0.83211679, 0.87591241,
        0.87591241, 0.88281376, 0.91209145, 0.8461394 , 0.89744553])}

In [57]:
#media e desvio padrao
print('Acuracia: {} (+/- {})'.format(scores_svm['test_accuracy'].mean(), scores_svm['test_accuracy'].std()) )
print('Kappa: {} (+/- {})'.format(kappa_svm['test_score'].mean(), kappa_svm['test_score'].std()))
print('Precision: {} (+/- {})'.format(scores_svm['test_precision'].mean(), scores_svm['test_precision'].std()) )
print('Recall: {} (+/- {})'.format(scores_svm['test_recall'].mean(), scores_svm['test_recall'].std()) )
print('F1: {} (+/- {})'.format(scores_svm['test_f1'].mean(), scores_svm['test_f1'].std()) )

Acuracia: 0.9382369455362156 (+/- 0.011112264189590697)
Kappa: 0.8764767505248748 (+/- 0.0222288257512527)
Precision: 0.9441548913998605 (+/- 0.015779616785802215)
Recall: 0.9320255474452555 (+/- 0.024674109994578954)
F1: 0.9377383810772703 (+/- 0.01181978482541035)


In [85]:
results_svm = {'classifier':'svm','params' : str(param_svm)}
results_svm.update({x.replace('test_',''): f'{scores_svm[x].mean()} (+/-{scores_svm[x].std():.5f})' for x in scores_svm.keys() if 'test' in x})
results_svm['kappa'] = f'{kappa_svm["test_score"].mean()} (+/-{kappa_svm["test_score"].std():.5f})'
results_svm

{'accuracy': '0.9382369455362156 (+/-0.01111)',
 'classifier': 'svm',
 'f1': '0.9377383810772703 (+/-0.01182)',
 'kappa': '0.8764767505248748 (+/-0.02223)',
 'params': "{'C': 0.1, 'kernel': 'linear'}",
 'precision': '0.9441548913998605 (+/-0.01578)',
 'recall': '0.9320255474452555 (+/-0.02467)'}

## Cross validation com random forest

In [89]:
clf = RandomForestClassifier(n_estimators=param_rf['n_estimators'])

In [90]:
clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=110)

In [91]:
pred = clf.predict(x_test)
accuracy_score(y_test, pred)

0.9562043795620438

In [96]:
metrics = ['accuracy','precision','recall','f1']

In [97]:
scores_rf = cross_validate(clf,X,Y,cv=10, scoring=metrics)

In [98]:
scores_rf

{'fit_time': array([1.16564941, 1.16439795, 1.16875958, 1.13512731, 1.14090657,
        1.14337206, 1.28414726, 1.43959498, 1.48014569, 1.13777518]),
 'score_time': array([0.02640772, 0.01931286, 0.01906466, 0.01934099, 0.01923418,
        0.01945257, 0.03648472, 0.07940078, 0.01923871, 0.02901793]),
 'test_accuracy': array([0.95255474, 0.97810219, 0.94890511, 0.94525547, 0.96715328,
        0.9379562 , 0.96336996, 0.98168498, 0.93406593, 0.95238095]),
 'test_f1': array([0.95373665, 0.97810219, 0.94736842, 0.94464945, 0.96727273,
        0.9390681 , 0.96296296, 0.98168498, 0.93333333, 0.95238095]),
 'test_precision': array([0.93055556, 0.97810219, 0.97674419, 0.95522388, 0.96376812,
        0.92253521, 0.97744361, 0.98529412, 0.94029851, 0.94890511]),
 'test_recall': array([0.97810219, 0.97810219, 0.91970803, 0.93430657, 0.97080292,
        0.95620438, 0.94890511, 0.97810219, 0.92647059, 0.95588235])}

In [99]:
kappa = make_scorer(cohen_kappa_score)

In [100]:
kappa_rf = cross_validate(clf, X, Y, cv=10, scoring=kappa)

In [101]:
kappa_rf

{'fit_time': array([1.22666359, 1.16382885, 1.14998412, 1.16100311, 1.15449524,
        1.13752079, 1.15536642, 1.14323926, 1.10935521, 1.14483762]),
 'score_time': array([0.01723742, 0.0181489 , 0.01718068, 0.0170927 , 0.01703858,
        0.01699924, 0.01686811, 0.01729798, 0.01708388, 0.01818776]),
 'test_score': array([0.91240876, 0.95620438, 0.90510949, 0.88321168, 0.94160584,
        0.88321168, 0.91941933, 0.94139116, 0.89009957, 0.919415  ])}

In [102]:
print('Acuracia: {} (+/- {})'.format(scores_rf['test_accuracy'].mean(), scores_rf['test_accuracy'].std()) )
print('Kappa: {} (+/- {})'.format(kappa_rf['test_score'].mean(), kappa_rf['test_score'].std()))
print('Precision: {} (+/- {})'.format(scores_rf['test_precision'].mean(), scores_rf['test_precision'].std()) )
print('Recall: {} (+/- {})'.format(scores_rf['test_recall'].mean(), scores_rf['test_recall'].std()) )
print('F1: {} (+/- {})'.format(scores_rf['test_f1'].mean(), scores_rf['test_f1'].std()) )

Acuracia: 0.9561428838801103 (+/- 0.015245210522089794)
Kappa: 0.9152076873811905 (+/- 0.02426173727407168)
Precision: 0.9578870482811093 (+/- 0.020799818259293938)
Recall: 0.9546586517818805 (+/- 0.02092067394135213)
F1: 0.9560559770125769 (+/- 0.015315931041907028)


In [103]:
results_rf = {'classifier':'rf','params' : str(param_rf)}
results_rf.update({x.replace('test_',''): f'{scores_rf[x].mean()} (+/-{scores_rf[x].std():.5f})' for x in scores_rf.keys() if 'test' in x})
results_rf['kappa'] = f'{kappa_rf["test_score"].mean()} (+/-{kappa_rf["test_score"].std():.5f})'
results_rf

{'accuracy': '0.9561428838801103 (+/-0.01525)',
 'classifier': 'rf',
 'f1': '0.9560559770125769 (+/-0.01532)',
 'kappa': '0.9152076873811905 (+/-0.02426)',
 'params': "{'n_estimators': 110}",
 'precision': '0.9578870482811093 (+/-0.02080)',
 'recall': '0.9546586517818805 (+/-0.02092)'}

# Salvar resultados

In [112]:
results = [results_rf,results_svm]
df = pd.DataFrame(results)
df = df[['classifier','params','accuracy','kappa','precision','f1','recall']]
df.rename(columns={'classifier':'clf','accuracy':'acc','precision':'prec'}, inplace=True)
df

,clf,params,acc,kappa,prec,f1,recall
0,rf,{'n_estimators': 110},0.9561428838801103 (+/-0.01525),0.9152076873811905 (+/-0.02426),0.9578870482811093 (+/-0.02080),0.9560559770125769 (+/-0.01532),0.9546586517818805 (+/-0.02092)
1,svm,"{'C': 0.1, 'kernel': 'linear'}",0.9382369455362156 (+/-0.01111),0.8764767505248748 (+/-0.02223),0.9441548913998605 (+/-0.01578),0.9377383810772703 (+/-0.01182),0.9320255474452555 (+/-0.02467)


In [118]:
os.mkdir(save+'teste_04')

In [120]:
df.to_csv(save+'teste_04/binario_aedes_aumentado_mfccs.csv',index=False)